In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from typing import Callable

# Cinemática inversa numérica

Sendo $x=f(\theta)$ o vetor das posições referentes aos ângulos ($\theta\in\mathbb{R}^n$), e $x_d \in\mathbb{R}^m$ as coordenadas desejadas, a solução será $\theta_d$ tal que $g(\theta_d)=x_d-f(\theta_d)=0$.

Pelo método de Newton-Raphson, para resolver uma equação do tipo $g(\theta)=0$ , se usa a expansão de Taylor de primeira ordem:

$g(\theta)=g(\theta_0)+\frac{\partial g}{\partial\theta}\Big|_{\theta_0}(\theta-\theta_0)~~$
$\overset{g(\theta)=0}{\longrightarrow}~~$
$\theta=\theta_0-\Big(\frac{\partial g}{\partial\theta}\Big|_{\theta_0}\Big)^{-1}g(\theta_0)$

Generalizando:

$\theta_{k+1}=\theta_k-\Big(\frac{\partial g}{\partial\theta}\Big|_{\theta_k}\Big)^{-1}g(\theta_k)$

Com isso:

$x_d=f(\theta_d)=f(\theta_0)+\underset{J(\theta_0)}{\boxed{\frac{\partial f}{\partial\theta}\Big|_{\theta_0}}}
\cdot\underset{\Delta\theta}{\boxed{(\theta_d-\theta_0)}}~,~~J(\theta_0)\in\mathbb{R}^{m\times n}~\Rightarrow~$
$J(\theta_0)\Delta\theta=x_d-f(\theta_0)~\Rightarrow$

$\theta_1=\theta_0+\Delta\theta$

Generalizando:

$\theta_{k+1}=\theta_k+\Delta\theta_k$

**Obs:** Sendo $f(\theta) = \big[f_1(\theta)~,~~f_2(\theta)~,~~\cdots~,~~f_n(\theta)\big]$:

$J(\theta)=$
$\begin{bmatrix}
\frac{\partial f_1}{\partial\theta_1}(\theta)&\cdots&\frac{\partial f_1}{\partial\theta_n}(\theta)\\
\vdots&\ddots&\vdots\\
\frac{\partial f_n}{\partial\theta_1}(\theta)&\cdots&\frac{\partial f_n}{\partial\theta_n}(\theta)
\end{bmatrix}$

## Caso para 1 dimensão

O caso para 1 dimensão se resume em encontrar um ângulo referente a uma posição em um círculo. Pode-se usar o método de Newton.

### Método de Newton

In [3]:
def nr(f: Callable[[float], float], df: Callable[[float], float], x: float, p: float) -> float:
    # p = chute inicial
    def g(w):
        return f(w) - x
    xl = p
    # critérios de parada
    N, e_stop = 50, 1e-8
    iter, e = 0, 1
    while iter < N-1 and e > e_stop:
        xn = xl - g(xl)/df(xl)
        e = xn
        xl = xn
        iter += 1

    return xn

In [4]:
R = 3

def f_1d(theta: float) -> float:
    return theta*R

def d_f_1d(theta: float) -> float:
    return R

In [5]:
theta_nr = nr(f_1d, d_f_1d, 2, 0.7)
print(f"Valor de theta encontrado para x = 2: {theta_nr}")
print(f"Valor de x: {f_1d(theta_nr)}")

Valor de theta encontrado para x = 2: 0.6666666666666667
Valor de x: 2.0


## Caso para 2 dimensões

$f=(x,y)=\Big(f_1(\theta_1,\theta_2),~f_2(\theta_1,\theta_2)\Big)=\Big(R_1\cos(\theta_1)+R_2\cos(\theta_2),~R_1\sin(\theta_1)+R_2\sin(\theta_2)\Big)$

$J=\begin{bmatrix}\frac{\partial f_1}{\partial\theta_1}&\frac{\partial f_1}{\partial\theta_2}\\\frac{\partial f_2}{\partial\theta_1}&\frac{\partial f_2}{\partial\theta_2}\end{bmatrix}=$
$\begin{bmatrix}-R_1\sin(\theta_1)&-R_2\sin(\theta_2)\\R_1\cos(\theta_1)&R_2\cos(\theta_2)\end{bmatrix}$

**Newton-Raphson:**

$\begin{bmatrix}
-R_1\sin(\theta_{1_k})&-R_2\sin(\theta_{2_k})\\
R_1\cos(\theta_{1_k})&R_2\cos(\theta_{2_k})
\end{bmatrix}$
$\begin{bmatrix}
\theta_{1_{k+1}}-\theta_{1_k}\\
\theta_{2_{k+1}}-\theta_{2_k}\\
\end{bmatrix}=$
$\begin{bmatrix}
x_{1_d}\\x_{2_d}
\end{bmatrix}-$
$\begin{bmatrix}
R_1\cos(\theta_{1_k})+R_2\cos(\theta_{2_k})\\
R_1\sin(\theta_{1_k})+R_2\sin(\theta_{2_k})
\end{bmatrix}$

$\begin{bmatrix}
\theta_{1_{k+1}}-\theta_{1_k}\\
\theta_{2_{k+1}}-\theta_{2_k}
\end{bmatrix}=
\begin{bmatrix}
-R_1\sin(\theta_{1_k})&-R_2\sin(\theta_{2_k})\\
R_1\cos(\theta_{1_k})&R_2\cos(\theta_{2_k})
\end{bmatrix}^{-1}
\begin{bmatrix}
x_{1_d} - \big(R_1\cos(\theta_{1_k})+R_2\cos(\theta_{2_k})\big)\\
x_{2_d} - \big(R_1\sin(\theta_{1_k})+R_2\sin(\theta_{2_k})\big)
\end{bmatrix}$

$\Theta = A^{-1}\cdot\Big[X-B\Big]$

In [6]:
def A_matrix_maker(R1, R2, theta1, theta2):
    return [[-R1*np.sin(theta1), -R2*np.sin(theta2)], [R1*np.cos(theta1), R2*np.cos(theta2)]]
def B_matrix_maker(R1, R2, theta1, theta2):
    return [R1*np.cos(theta1)+R2*np.cos(theta2), R1*np.sin(theta1)+R2*np.sin(theta2)]

In [7]:
def nr_2d(R1: float, R2: float, theta1: float, theta2: float, xd: list) -> tuple:
    xd = np.asarray(xd)
    mod = np.sqrt(xd[0]**2 + xd[1]**2)
    if mod > R1+R2 or mod < abs(R1-R2):
        raise ValueError("Não alcançável")
    loop = True
    iter = 0
    while loop:
        iter += 1
        A_matrix = np.asarray(A_matrix_maker(R1, R2, theta1, theta2))
        B_matrix = np.asarray(B_matrix_maker(R1, R2, theta1, theta2))
        Theta_vector = np.matmul(np.linalg.inv(A_matrix), xd - B_matrix)
        theta1, theta2 = Theta_vector[0] + theta1, Theta_vector[1] + theta2
        x = np.asarray(B_matrix_maker(R1, R2, theta1, theta2))
        loop = iter < 50 and not np.allclose(xd, x, rtol=1e-10)
    return theta1, theta2, x

In [8]:
theta1, theta2 = (-np.pi/4, np.pi/4)
R1, R2 = 3, 2.5

for i in range(50):
    xd = np.random.uniform(1, 5, [1, 2])[0]
    try:
        _, _, x = nr_2d(R1, R2, theta1, theta2, xd)
        print(f"x desejado: {xd}  x encontrado: {x}")
    except (ValueError, np.linalg.LinAlgError) as e:
        print(e)

x desejado: [1.83306087 1.08330911]  x encontrado: [1.83306087 1.08330911]
x desejado: [1.71709992 1.99433   ]  x encontrado: [1.71709992 1.99433   ]
x desejado: [2.16726946 2.82245192]  x encontrado: [2.16726946 2.82245192]
x desejado: [1.12262952 1.62493023]  x encontrado: [1.12262952 1.62493023]
x desejado: [3.6971612  1.23671645]  x encontrado: [3.6971612  1.23671645]
x desejado: [4.51307717 1.17566602]  x encontrado: [4.51307717 1.17566602]
Não alcançável
x desejado: [3.0496628  3.92108351]  x encontrado: [3.0496628  3.92108351]
x desejado: [3.59402238 1.92979866]  x encontrado: [3.59402238 1.92979866]
x desejado: [2.09270755 2.31513893]  x encontrado: [2.09270755 2.31513893]
Não alcançável
Não alcançável
x desejado: [2.1968991  1.54259336]  x encontrado: [2.1968991  1.54259336]
x desejado: [2.19900171 2.30873851]  x encontrado: [2.19900171 2.30873851]
Não alcançável
x desejado: [2.55366758 1.08205598]  x encontrado: [2.55366758 1.08205598]
Não alcançável
x desejado: [3.51418277 1